In [6]:
# %load_ext autoreload
# %autoreload 2

# Calculate DV01 and roll-down for various swap strategies



In [7]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# pretty-preview of dataframes (does not work for me)
# from jupyter_datatables import init_datatables_mode # pip install jupyter-datatables


In [8]:
from financepy.utils import *
from financepy.products.rates import *
from financepy.market.curves import *
from financepy.utils.day_count import DayCount, DayCountTypes

import financepy.products.rates.ibor_curve_risk_engine as re
from financepy.products.rates.ibor_benchmarks_report import dataframe_to_benchmarks

### Print dataframes a certain way

In [9]:
pd.set_option('display.max_rows', None,
                    'display.max_columns', None,
                    'display.float_format', lambda x:f'{x:.4f}'
                    )

# pretty-preview of dataframes (does not work for me)
# init_datatables_mode()

### Function to plot a curve

In [10]:

def plotCurve(curve, start_date, tmax, instr_mat_dates_or_tenor = None, title = ''):
    years = np.linspace(1/365, tmax, int(365*tmax)//30)
    dates = start_date.add_years(years)
    zero_rates = curve.zero_rate(dates)
    on_fwd_rates = curve.fwd(dates)

    ys_monthly = np.arange(1,tmax*12+1)/12
    monthly_dates = start_date.add_years(ys_monthly)

    if instr_mat_dates_or_tenor is not None:
        # Will plot term forward rates
        # instr_mat_dates_or_tenor could be a list of curve-building
        # isntrument maturities in which case term fwd rates go from the last instrument maturiy date that is less
        # than the plotting date to the plotting date. Or it could be a tenor so that fwd rates cover that tenor 
        # i.e. the term fwd rate for d covers [max(d-tenor,start_date), d]
        # tenor should be a positive tenor

        if isinstance(instr_mat_dates_or_tenor, str):
            neg_tenor = '-' + instr_mat_dates_or_tenor
            start_fwd_dates = [d.add_tenor(neg_tenor) for d in dates]
            start_fwd_dates = [d if d > start_date else start_date for d in start_fwd_dates]
            fwd_rate_label = f'term fwd rates for [d-{instr_mat_dates_or_tenor},d]'
        else:
            instr_mat_dates_or_tenor = [start_date] + instr_mat_dates_or_tenor
            start_fwd_dates = [ max([md  for md in instr_mat_dates_or_tenor if md < d]) for d in dates]
            fwd_rate_label = 'term fwd rates from prev instr mtrty'

        term_fwd_rates = curve.fwd_rate(start_fwd_dates, dates)

    plt.figure(figsize=(8,6))
    plt.plot(years, zero_rates*100, '-', label="zero rates")
    plt.plot(years, on_fwd_rates*100, '-', label = "ON fwd rates")

    if instr_mat_dates_or_tenor is not None:
        plt.plot(years, term_fwd_rates*100, '.', label = fwd_rate_label)

    plt.xlabel("Times in years")
    plt.ylabel("Rates (%) - See Legend")
    plt.title(title)
    plt.legend()

### Load csv with curve bechmarks

In [11]:
filename = './data/bms_GBP_SONIA_20220901_10y_off.csv'
dfbm = pd.read_csv(filename, index_col = 0)
dfbm['base_date']=pd.to_datetime(dfbm['base_date'], errors = 'ignore', format = '%d/%m/%Y')
dfbm['start_date']=pd.to_datetime(dfbm['start_date'], errors = 'ignore', format = '%d/%m/%Y')
dfbm['maturity_date']=pd.to_datetime(dfbm['maturity_date'], errors = 'ignore', format = '%d/%m/%Y')
dfbm.head()


FileNotFoundError: [Errno 2] No such file or directory: './data/bms_GBP_SONIA_20220901_10y_off.csv'

### Convert to benchmarks

In [10]:

valuation_date = date.from_datetime(dfbm.loc[0,'base_date'])
cal = CalendarTypes.UNITED_KINGDOM
bms = dataframe_to_benchmarks(dfbm, asof_date=valuation_date,calendar_type=cal)
depos = bms['IborDeposit']
fras = bms['IborFRA']
swaps = bms['IborSwap']

fras.sort( key = lambda fra:fra.maturity_dt)


### Remove MPC FRAs to simplify the curve

In [ ]:
remove_mpc_fras = True
if remove_mpc_fras:
    fras_to_use = [f for f in fras if date.datediff(f.start_dt,f.maturity_dt) > 60]
    fras = fras_to_use
len(fras)

## Make a shorter curve

In [ ]:
last_year = 30
swaps_to_use = [s for s in swaps if DayCount(DayCountTypes.SIMPLE).year_frac(s.effective_dt,s.maturity_dt)[0] <=  last_year]
swaps = swaps_to_use
len(swaps)

### Calibrate and plot the curve

In [ ]:
curve = IborSingleCurve(valuation_date, depos, fras, swaps, InterpTypes.PCHIP_ZERO_RATES)
plotCurve(curve, valuation_date, 20, instr_mat_dates_or_tenor='1Y')

### Some common params

In [15]:
spot_days = 2
settlement_date = valuation_date.add_weekdays(spot_days)
fixedDCCType = DayCountTypes.ACT_365F
fixedFreqType = FrequencyTypes.ANNUAL

### Look at different outright swaps 

In [18]:
# Outrights all normalized to have dv01 ~ 15

outrights = {}
outr_labels =[]
for n in np.arange(1,16):
    s = IborSwap(settlement_date, f'{n}Y', SwapTypes.PAY, 1*g_percent, fixedFreqType, fixedDCCType, cal_type=cal, notional = 10_000 * 15.0/n)
    s.set_fixed_rate_to_atm(valuation_date, curve)    
    outrights[f'outr{n}'] = s


### Define some strategies

In [19]:
# Steepeners using spot swaps
swap5y = IborSwap(settlement_date, "5Y", SwapTypes.RECEIVE, 1*g_percent, fixedFreqType, fixedDCCType, cal_type=cal, notional = 30_000)
swap15y = IborSwap(settlement_date, "15Y", SwapTypes.PAY, 1*g_percent, fixedFreqType, fixedDCCType, cal_type=cal, notional = 10_000)

steep_spot_5_15 = {
    'spot5y':swap5y,
    'spot15y':swap15y,
}


In [20]:
# Steepener using fwd swaps
swap0y5y = IborSwap(settlement_date, "5Y", SwapTypes.RECEIVE, 1*g_percent, fixedFreqType, fixedDCCType, cal_type=cal, notional = 20_000)
swap5y10y = IborSwap(settlement_date.add_tenor('5Y'), "10Y", SwapTypes.PAY, 1*g_percent, fixedFreqType, fixedDCCType, cal_type=cal, notional = 10_000)
swap0y5y.set_fixed_rate_to_atm(valuation_date, curve)
swap5y10y.set_fixed_rate_to_atm(valuation_date, curve)
steep_fwd_5_15 = {
    'fwd0y5y':swap0y5y,
    'fwd5y10y':swap5y10y,
    }


In [ ]:

# Fly using spot swaps
swap9y = IborSwap(settlement_date, "9Y", SwapTypes.PAY, 1*g_percent, fixedFreqType, fixedDCCType, cal_type=cal, notional = 10_000)
swap10y = IborSwap(settlement_date, "10Y", SwapTypes.RECEIVE, 1*g_percent, fixedFreqType, fixedDCCType, cal_type=cal, notional = 20_000)
swap11y = IborSwap(settlement_date, "11y", SwapTypes.PAY, 1*g_percent, fixedFreqType, fixedDCCType, cal_type=cal, notional = 10000)

fly_spot_9_10_11 = {
    'spot9y' : swap9y,
    'spot10y' : swap10y,
    'spot11y' : swap11y,
}    

In [23]:
# Fly using forward swaps
swap9y1y = IborSwap(settlement_date.add_tenor('9Y'), "1Y", SwapTypes.RECEIVE, 1*g_percent, fixedFreqType, fixedDCCType, cal_type=cal, notional = 10_000)
swap10y1y = IborSwap(settlement_date.add_tenor('10Y'), "1Y", SwapTypes.PAY, 1*g_percent, fixedFreqType, fixedDCCType, cal_type=cal, notional = 10_000)

fly_fwd_9y1y_10y1y = {
    'fwd9y1y' : swap9y1y,
    'fwd10y1y' : swap10y1y,
}
for s in fly_fwd_9y1y_10y1y.values():
    s.set_fixed_rate_to_atm(valuation_date, curve)

### Calculate rolldown and carry

In [ ]:
trade_to_use = fly_spot_9_10_11 # outrights # fly_spot_9_10_11 # fly_fwd_9y1y_10y1y# fly_spot_9_10_11
rolldown_tenor = '1Y'
last_date = swaps[-1].maturity_dt
risk_res = re.carry_rolldown_report(
    curve, grid_last_date = last_date, grid_bucket_tenor = rolldown_tenor, 
    trades = trade_to_use.values(), trade_labels=list(trade_to_use.keys()), )
df_roll = risk_res[1]
df_roll

In [ ]:
df_roll[[c for c in df_roll.columns if c.startswith('ROLL')]].sum()

In [ ]:
dv01_cols = [c for c in df_roll.columns if c.startswith('DV01')]
nonzero_dv01 = (df_roll[dv01_cols] != 0).any(axis = 1)
df_dv01 = df_roll.loc[nonzero_dv01]
df_dv01.plot.bar('bucket_label', dv01_cols)

In [ ]:
df_roll.plot(x='maturity_date', y='market_rate', marker = '.')

### Wrap the above into a function

In [ ]:
def plot_metric_for_strategy(curve, strategy, metric = 'DV01'):
    rolldown_tenor = '1Y'
    last_date = curve.used_swaps[-1].maturity_dt
    risk_res = re.carry_rolldown_report(
    curve, grid_last_date = last_date, grid_bucket_tenor = rolldown_tenor, 
    trades = strategy.values(), trade_labels=list(strategy.keys()), )
    df_roll = risk_res[1]

    metric_cols = [c for c in df_roll.columns if c.startswith(metric)]
    nonzero_metric = (df_roll[metric_cols] != 0).any(axis = 1)
    df_metric = df_roll.loc[nonzero_metric]
    df_metric.plot.bar('bucket_label', metric_cols)

    print(df_metric[metric_cols].sum())

#Test (with somewhat convoluted way to just get first 3 outrights)
plot_metric_for_strategy(curve, dict(list(outrights.items())[:3]), metric='DV01')


In [ ]:
metric_to_plot = 'DV01'
# strategies to plot
strategies_to_plot = [
#    steep_spot_5_15,
#    steep_fwd_5_15,
    fly_spot_9_10_11,
    fly_fwd_9y1y_10y1y,
]

for s in strategies_to_plot:
    plot_metric_for_strategy(curve, s,  metric=metric_to_plot)